In [ ]:
# collecting data by webscraping
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import numpy as np
import unicodedata
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
url = requests.get(static_url).text
soup = BeautifulSoup(url,'html5lib')
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass 
  
html_tables = soup.find_all('table')
first_launch_table = html_tables[2]
x = first_launch_table.find('tr').find_all('th')
column_names = []
for y in x :
    name = ' '.join(y.stripped_strings)
    if name :
        column_names.append(name)
column_names[2].strip('[b]')
column_names[4].strip(' [c]')
launch_dict= dict.fromkeys(column_names)
del launch_dict['Date and time ( UTC )']
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload [c]'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version, Booster [b]']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]
extracted_row = 0
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            
            # Date value
            fulldate = list(row[0].strings)
            date = fulldate[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time value
            time = fulldate[1].strip()
            launch_dict['Time'].append(time)
              
            # Booster version
            z = list(row[1].strings)[0:3:2]
            bv = ' '.join(z) 

            launch_dict['Version, Booster [b]'].append(bv) 
            
            # Launch Site
            launch_site1 = list(row[2].strings)[0:3:2]
            launch_site = ' '.join(launch_site1) 
            launch_dict['Launch site'].append(launch_site)
            
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload [c]'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].text.strip()
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer= row[6].text.strip()
            launch_dict['Customer'].append(customer) 
        
            
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = list(row[8].strings)[0].strip()
            launch_dict['Booster landing'].append(booster_landing)

df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df.rename(columns={'Version, Booster [b]':'Version Booster'},inplace=True)
df.rename(columns={'Payload [c]':'Payload'},inplace=True)
df.to_csv('spacex_web_scrabed.csv',index=False)
            

In [ ]:
# collecting data by APIs
# that is an additional way of collecting data with web scraping
import requests
import pandas as pd
import numpy as np
import datetime
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])
static_json_url='https://api.spacexdata.com/v4/launches/past'
response = requests.get(static_json_url).json()
data = pd.json_normalize(response)
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

data['date'] = pd.to_datetime(data['date_utc']).dt.date

data = data[data['date'] <= datetime.date(2020, 11, 13)]
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []
getBoosterVersion(data)
getLaunchSite(data)
getPayloadData(data)
getCoreData(data)
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}
df = pd.DataFrame(launch_dict)
data_falcon9 = df[df['BoosterVersion'] == 'Falcon 9']
data_falcon9.loc[:,'FlightNumber'] = range(1,91,1)
data_falcon9['PayloadMass'].replace(np.nan,data_falcon9['PayloadMass'].mean(),inplace=True)
data_falcon9.head()
data_falcon9.to_csv('dataset_part_1.csv', index=False)


In [ ]:
#data wrangling
import pandas as pd
import numpy as np
df=pd.read_csv('dataset_part_1.csv')
outcome_landing = df['Outcome'].value_counts()
bad_outcomes = list(outcome_landing.keys()[[1,3,5,6,7]])
df['class'] = np.where(df['Outcome'].isin(bad_outcomes), 0, 1)
#df['class'].mean()
#df.groupby('Orbit')['Outcome'].value_counts().to_frame()
#df['Outcome'].value_counts()
df.head()
df.to_csv('dataset_part_2.csv',index=False)


In [ ]:
#EDA with SQL
import pandas as pd
import csv, sqlite3
%load_ext sql
%sql sqlite:///my_data1.db
con = sqlite3.connect("my_data1.db")
cur = con.cursor()
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null
%sql select * from SPACEXTABLE limit 5
%%sql
select distinct(Launch_Site) from SPACEXTABLE 
select * from SPACEXTABLE where Launch_Site like 'CCA%' limit 5
select Customer, sum(PAYLOAD_MASS__KG_) as total_payload_mass from SPACEXTABLE where Customer = 'NASA (CRS)'
select Booster_Version, avg(PAYLOAD_MASS__KG_) as avg_payload_mass from SPACEXTABLE where Booster_Version = 'F9 v1.1'
select min(Date),Landing_Outcome from SPACEXTABLE where Landing_Outcome like 'Success%'
select Booster_Version,PAYLOAD_MASS__KG_,Landing_Outcome  from SPACEXTABLE where Landing_Outcome = 'Success (drone ship)'  AND PAYLOAD_MASS__KG_ Between 4000 and 6000
select Mission_Outcome, count(Mission_Outcome) from SPACEXTABLE group by Mission_Outcome
select Booster_version,PAYLOAD_MASS__KG_ from SPACEXTABLE where PAYLOAD_MASS__KG_ = (select MAX(PAYLOAD_MASS__KG_) from SPACEXTABLE)
select Date, substr(Date,6,2) as month ,Landing_Outcome, Booster_Version , Launch_Site from SPACEXTABLE where Landing_Outcome like 'Failure (drone ship)' AND substr(Date,1,4)='2015'
select Landing_Outcome, count(Landing_Outcome) as count from SPACEXTABLE where Date BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY Landing_Outcome ORDER BY Count DESC;


In [ ]:
#SpaceX prediction 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split,GridSearchCV 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
from sklearn.metrics import log_loss,jaccard_score,f1_score
pd.set_option('display.max_columns', None)
df=pd.read_csv("dataset_part_2 (1).csv")
df = df[['PayloadMass','Orbit','LaunchSite','Serial','Flights','GridFins','Reused','Legs','Block','ReusedCount','Class']]
df[['GridFins','Reused','Legs','Block']] = df[['GridFins','Reused','Legs','Block']].astype(int)
df = pd.get_dummies(df).astype(int)
x = df.drop(columns='Class').values
y = df['Class'].values 
x = preprocessing.StandardScaler().fit(x).transform(x.astype(int))
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=4)

df.head()

df


In [ ]:
#Decision Tree
param_grid = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}
model = DecisionTreeClassifier()
grid_search_decision_tree = GridSearchCV(model, param_grid, cv=5)
grid_search_decision_tree.fit(x_train,y_train)
print("Best Parameters: ", grid_search_decision_tree.best_params_)
print("Best Score: ", grid_search_decision_tree.best_score_)
best_model = grid_search_decision_tree.best_estimator_
print(best_model)
yhat=grid_search_decision_tree.predict(x_test)
yhat[0:10],y_test[0:10]




In [ ]:
#KNN
param_grid = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}
model = KNeighborsClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(x_train,y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
print(best_model)
yhat=grid_search.predict(x_test)
yhat[0:10],y_test[0:10]


In [ ]:
#SVM
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}
model = SVC()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(x_train,y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
print(best_model)
yhat=grid_search.predict(x_test)
yhat[0:10],y_test[0:10]





In [ ]:
#Logistic Regression
param_grid = {'C': [0.01,0.1, 1, 10, 100], 'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}
model = LogisticRegression()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(x_train,y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
print(best_model)
yhat=grid_search.predict(x_test)
yhat[0:10],y_test[0:10]


In [34]:
#assuming new data to precict its class
pd.set_option('display.max_columns', None)
df=pd.read_csv("dataset_part_2 (1).csv")
df1 = df.drop(columns='Class')
#last row of df1 is the row that hold the new data we want to have prediction on it
df1.loc[len(df1)] = {'FlightNumber':91,	'Date':2013-12-1 ,	'BoosterVersion':'Falcon 9',	'PayloadMass':986,	'Orbit':'LEO',	'LaunchSite':'CCAFS SLC 40','Outcome':'None None',	'Flights':2,	'GridFins':1,	'Reused':0,	'Legs':1,	'LandingPad':'NaN',	'Block':2,	'ReusedCount':5,	'Serial':'B1060',	'Longitude':-80.577366,	'Latitude':28.561857}
df1 = df1[['PayloadMass','Orbit','LaunchSite','Serial','Flights','GridFins','Reused','Legs','Block','ReusedCount']]
df1['Block'] = df1['Block'].astype(int)
df1 = pd.get_dummies(df1).astype(int).values
df1 = preprocessing.StandardScaler().fit(df1).transform(df1.astype(int))
k = df1[90]
k = k.reshape(1, -1)
#Decision Tree
yhat=grid_search_decision_tree.predict(k)
yhat
# it will have successfull landing



array([1])